In [17]:
from qiskit_nature.units import *   #ANGSTROM
from qiskit_nature.second_q.drivers import *    #PySCFDriver
from qiskit_nature.second_q.mappers import * #JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.problems import EigenstateResult
from qiskit_nature.second_q.circuit.library import *  #Ansatz, HF
#from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options
from qiskit_algorithms.optimizers import * 
from qiskit_algorithms import VQE
from braket.tracking import Tracker
from qiskit_braket_provider import *
from qiskit_aer import StatevectorSimulator
from qiskit.primitives import Estimator
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_algorithms import AdaptVQE
import numpy as np
import pandas as pd

In [18]:
def co2_sim(z,y):
    driver = PySCFDriver(atom=f"O 0 {y} -{z}; C 0 0 0; O 0 {y} {z}", basis="sto-6g")
    estrutura_inicial = driver.run() #Criamos a estrutura do nosso sistema

    espaco_target_minimal = ActiveSpaceTransformer(4,4) #Active Space para o H2
    estrutura_minimal = espaco_target_minimal.transform(estrutura_inicial)
    mapa_minimal = estrutura_minimal.get_tapered_mapper(BravyiKitaevMapper())

    ansatz = UCCSD(
    estrutura_minimal.num_spatial_orbitals,
    estrutura_minimal.num_particles,
    mapa_minimal,
    initial_state=HartreeFock(
        estrutura_minimal.num_spatial_orbitals,
        estrutura_minimal.num_particles,
        mapa_minimal))


    vqe_solver = VQE(Estimator(), ansatz, SLSQP())
    vqe_solver.initial_point = [0.0] * ansatz.num_parameters


    calculo = GroundStateEigensolver(mapa_minimal, vqe_solver)
    resultado = calculo.solve(estrutura_minimal)
    return resultado

In [14]:
raio = np.arange(1,1.3,0.01)
theta = np.arange(np.pi/4,np.pi/2 + np.pi/40, np.pi/40)
par = [[r,o] for r in raio for o in theta]

In [19]:
dados = [[a[0]*np.sin(a[1]), a[0]*np.cos(a[1]), co2_sim(a[0]*np.sin(a[1]),a[0]*np.cos(a[1]))] for a in par]

MissingOptionalLibraryError: "The 'pyscf' library is required to use 'PySCFDriver'.  See https://pyscf.org/install.html."